In [2]:
import gmaps
import pandas as pd
import requests 
import json
import matplotlib.pyplot as plt

In [3]:
%matplotlib notebook

In [6]:
# from ipython_config import gkey
# print(gkey)

In [7]:
data = pd.read_csv('Hospital_General_Information.csv')
hospital_df = pd.DataFrame(data)
hospital_df.columns
hospital_df.columns
NC = hospital_df.loc[hospital_df['State']=='NC']
NC = hospital_df

In [8]:
with open('beds.json') as jsonfile:
    video_json = json.load(jsonfile)

In [9]:
bed_df = pd.DataFrame(video_json)
bed_df.columns
bed_df=bed_df[['hospital_bed_count','medicare_provider_number' ]]
bed_df = bed_df.rename(columns={'medicare_provider_number':'Facility ID'})
bed_df.head()

,hospital_bed_count,Facility ID
0,99,010007
1,362,010011
2,252,010016
3,106,010018
4,349,010024


In [17]:
del NC['Hospital overall rating footnote']
del NC['Mortality national comparison footnote']
del NC['Safety of care national comparison footnote']
del NC['Readmission national comparison footnote']
del NC['Patient experience national comparison footnote']
del NC['Efficient use of medical imaging national comparison footnote']
del NC['Effectiveness of care national comparison footnote']
del NC['Timeliness of care national comparison footnote']
del NC['Address']
del NC['City']
del NC['Hospital Ownership']
del NC['Phone Number']
NC.head(15)
df = pd.merge(NC, bed_df, on = 'Facility ID', how = 'left')


KeyError: 'Hospital overall rating footnote'

In [79]:
df2 = df.loc[df['Hospital overall rating']!='Not Available']
df2 =df2.astype({'Hospital overall rating':'int64'})
states = df2['State'].value_counts()
df.head()

,Facility ID,Facility Name,State,ZIP Code,County Name,Hospital Type,Emergency Services,Meets criteria for meaningful use of EHRs,Hospital overall rating,Mortality national comparison,Safety of care national comparison,Readmission national comparison,Patient experience national comparison,Effectiveness of care national comparison,Timeliness of care national comparison,Efficient use of medical imaging national comparison,Location,hospital_bed_count
0,031309,SAGE MEMORIAL HOSPITAL,AZ,86505,APACHE,Critical Access Hospitals,False,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,NaN,25
1,044021,WOODRIDGE BEHAVIORAL CENTER,AR,72301,CRITTENDEN,Psychiatric,False,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,POINT (-90.175858 35.152962),NaN
2,100277,DOUGLAS GARDENS HOSPITAL,FL,33137,MIAMI-DADE,Acute Care Hospitals,False,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,POINT (-80.191767 25.822972),32
3,104078,SUNCOAST BEHAVIORAL HEALTH CENTER,FL,34210,MANATEE,Psychiatric,False,NaN,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,POINT (-82.612355 27.460155),NaN
4,130063,TREASURE VALLEY HOSPITAL,ID,83704,ADA,Acute Care Hospitals,False,NaN,Not Available,Not Available,Not Available,Not Available,Above the national average,Not Available,Not Available,Not Available,POINT (-116.290332 43.611289),NaN


In [69]:
spare = df2.loc[(df2['County Name']=='WAKE')&(df2['State']=='NC')]
spare.dtypes

Facility ID                                             object
Facility Name                                           object
State                                                   object
ZIP Code                                                 int64
County Name                                             object
Hospital Type                                           object
Emergency Services                                        bool
Meets criteria for meaningful use of EHRs               object
Hospital overall rating                                  int64
Mortality national comparison                           object
Safety of care national comparison                      object
Readmission national comparison                         object
Patient experience national comparison                  object
Effectiveness of care national comparison               object
Timeliness of care national comparison                  object
Efficient use of medical imaging national comparison   

In [105]:
averages ={}
for st in states.index:
    hold_df = df2.loc[df2['State']==st]
    counties = hold_df['County Name'].value_counts()
    averages.update( {st : {}} )
    averages[st].update({'County':[],'Average Rating':[],'Emergency Services':[]})
    for cnt in counties.index:
        county = hold_df.loc[(df['County Name'] == cnt)]
        avg = county.mean()
        averages[st]['County'].append(cnt)
        averages[st]['Average Rating'].append(avg['Hospital overall rating'])
        averages[st]['Emergency Services'].append(avg['Emergency Services'])    

In [106]:
sums = {}
df3 = df[['County Name','State','hospital_bed_count']]
df3 = df3.dropna()
df3 = df3.astype({'hospital_bed_count':'int64'})
for st in states.index:
    hold_df = df3.loc[df3['State']==st]
    counties = hold_df['County Name'].value_counts()
    sums.update( {st : {}} )
    sums[st].update({'County':[],'County Beds':[]})
    for cnt in counties.index:
        county = hold_df.loc[(df['County Name'] == cnt)]
        totals = county.sum()
        sums[st]['County'].append(cnt)
        sums[st]['County Beds'].append(totals['hospital_bed_count'])  

In [107]:
counts = {}
for st in states.index:
    hold_df = df.loc[df['State']==st]
    counties = hold_df['County Name'].value_counts()
    counts.update( {st : {}} )
    counts[st].update({'County':[],'Hospitals':[]})
    for cnt in counties.index:
        county = hold_df.loc[(df['County Name'] == cnt)]
        totals = county.count()
        counts[st]['County'].append(cnt)
        counts[st]['Hospitals'].append(totals['Facility ID']) 

In [100]:
CA = pd.DataFrame(counts['CA'])
CA.head()

,County,Hospitals
0,LOS ANGELES,90
1,ORANGE,28
2,SAN BERNARDINO,22
3,SAN DIEGO,21
4,RIVERSIDE,17


In [116]:
hold_df = pd.DataFrame({'County':[],'State':[],'Average Rating':[],'Hospitals':[],
                        'Emergency Services':[],'County Beds':[]})
for st in states.index:
    count_state = pd.DataFrame(counts[st])
    avg_state = pd.DataFrame(averages[st])
    sum_state = pd.DataFrame(sums[st])
    state_df = pd.merge(count_state,avg_state,on = 'County',how= 'outer')
    state_df = pd.merge(state_df,sum_state,on = 'County',how='outer')
    state_df['State'] = st
    final_df = pd.concat([hold_df,state_df])
    hold_df = final_df
final_df.head()
final_df.to_csv('CountyHospitalData.csv')

In [117]:
NC = final_df.loc[final_df['State']=='NC']
NC['ID'] = (NC['County'].str.title() + ', '+ NC['State'])
NC.to_csv('NCHospitalData.csv')

C:\Users\littl\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords,
    "rankby": "distance",
    "type": "hospital",
    "key": gkey,
}
